In [119]:
import pandas as pd
import csv
from collections import Counter

In [120]:
amazon_sale = pd.read_csv('data/Amazon Sale Report.csv', sep=',')

/var/folders/pt/wl3z1yr91qbfxjcs0bnz5c0r0000gn/T/ipykernel_87031/2294166161.py:1: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  amazon_sale = pd.read_csv('data/Amazon Sale Report.csv', sep=',')


In [121]:
print(amazon_sale.columns)
print(amazon_sale.shape)

Index(['index', 'Order ID', 'Date', 'Status', 'Fulfilment', 'Sales Channel ',
       'ship-service-level', 'Style', 'SKU', 'Category', 'Size', 'ASIN',
       'Courier Status', 'Qty', 'currency', 'Amount', 'ship-city',
       'ship-state', 'ship-postal-code', 'ship-country', 'promotion-ids',
       'B2B', 'fulfilled-by', 'Unnamed: 22'],
      dtype='object')
(128975, 24)


In [122]:
# Counter(amazon_sale['ASIN'].tolist())

In [148]:
#total number of items
print(amazon_sale['ASIN'].nunique())
print(amazon_sale['Order ID'].nunique())
#buy more than one item in a transaction
duplicate_orders = amazon_sale[amazon_sale.duplicated(subset ='Order ID')]

#items that have enough appearance in the item sets
frequent_items_ids = duplicate_orders[duplicate_orders['ASIN'].map(duplicate_orders['ASIN'].value_counts()) > 20]['Order ID']
interested_sales = amazon_sale[amazon_sale['Order ID'].isin(frequent_items_ids)][['Order ID', 'ASIN']]
print(interested_sales['ASIN'].unique().size)

7190
120378
88


In [149]:
basket = interested_sales.groupby(['Order ID', 'ASIN']).size().reset_index(name = 'Count')
print(basket)

                Order ID        ASIN  Count
0    171-1597237-7059516  B09SDX13G7      1
1    171-1597237-7059516  B09SDXRYBG      1
2    171-1614212-0216336  B09SDXRYBG      1
3    171-1614212-0216336  B09SDY9SQ6      1
4    171-1687588-5890754  B09SDXFFQ1      1
..                   ...         ...    ...
223  408-5454943-9201928  B09SDXRYBG      1
224  408-7304435-2235565  B08RP3LHNB      1
225  408-7304435-2235565  B09SDY8DCT      1
226  408-7522491-5998736  B09SDXFFQ1      1
227  408-7522491-5998736  B09SDYL6L1      1

[228 rows x 3 columns]


In [150]:
# basket = amazon_sale.groupby(['Order ID', 'ASIN']).size().reset_index(name = 'Count')

In [151]:
my_basket = (basket.groupby(['Order ID', 'ASIN'])['Count'].sum().unstack().reset_index().fillna(0).set_index('Order ID'))

In [152]:
print(my_basket)

ASIN                 B06Y5CFQSQ  B077M6KWYJ  B0783Q5HMF  B07GRVC75J  \
Order ID                                                              
171-1597237-7059516         0.0         0.0         0.0         0.0   
171-1614212-0216336         0.0         0.0         0.0         0.0   
171-1687588-5890754         0.0         0.0         0.0         0.0   
171-3283789-8893159         0.0         0.0         0.0         0.0   
171-3494553-7071538         0.0         0.0         0.0         0.0   
...                         ...         ...         ...         ...   
408-4128435-6095562         0.0         0.0         0.0         0.0   
408-4521661-0278767         0.0         0.0         0.0         0.0   
408-5454943-9201928         0.0         0.0         0.0         0.0   
408-7304435-2235565         0.0         0.0         0.0         0.0   
408-7522491-5998736         0.0         0.0         0.0         0.0   

ASIN                 B07HWMZ61T  B07R51FQDF  B07ZYQQZ28  B081X4S825  \
Order

In [153]:
def encode_data(datapoint):
 if datapoint <= 0:
     return 0
 else:
     return 1

In [154]:
my_basket = my_basket.applymap(encode_data)
print(my_basket.head())
print(my_basket.isna().sum())

ASIN                 B06Y5CFQSQ  B077M6KWYJ  B0783Q5HMF  B07GRVC75J  \
Order ID                                                              
171-1597237-7059516           0           0           0           0   
171-1614212-0216336           0           0           0           0   
171-1687588-5890754           0           0           0           0   
171-3283789-8893159           0           0           0           0   
171-3494553-7071538           0           0           0           0   

ASIN                 B07HWMZ61T  B07R51FQDF  B07ZYQQZ28  B081X4S825  \
Order ID                                                              
171-1597237-7059516           0           0           0           0   
171-1614212-0216336           0           0           0           0   
171-1687588-5890754           0           0           0           0   
171-3283789-8893159           0           0           0           0   
171-3494553-7071538           0           0           0           0   

ASIN

In [155]:
import mlxtend as ml
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [163]:
new_items = apriori(my_basket, min_support=0.02, use_colnames=True)

/Users/eho/anaconda3/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [164]:
print(new_items)

     support                              itemsets
0   0.020408                          (B081X4S825)
1   0.020408                          (B0894Y4PNG)
2   0.020408                          (B09KXT4VG7)
3   0.030612                          (B09NDKQD88)
4   0.234694                          (B09NQ44RNV)
5   0.040816                          (B09SDX13G7)
6   0.326531                          (B09SDXFFQ1)
7   0.051020                          (B09SDXG1CP)
8   0.071429                          (B09SDXQ3X9)
9   0.214286                          (B09SDXRYBG)
10  0.020408                          (B09SDXSQ33)
11  0.030612                          (B09SDXTRS9)
12  0.040816                          (B09SDXWNPZ)
13  0.061224                          (B09SDY51NS)
14  0.265306                          (B09SDY8DCT)
15  0.030612                          (B09SDY9SQ6)
16  0.020408                          (B09SDYHWLX)
17  0.040816                          (B09SDYL6L1)
18  0.030612                   

In [165]:
new_rules = association_rules(new_items, metric='lift', min_threshold=0.5)
print(new_rules.shape[0])
print(new_rules)

46
                 antecedents               consequents  antecedent support  \
0               (B09NQ44RNV)              (B081X4S825)            0.234694   
1               (B081X4S825)              (B09NQ44RNV)            0.020408   
2               (B0894Y4PNG)              (B09NQ44RNV)            0.020408   
3               (B09NQ44RNV)              (B0894Y4PNG)            0.234694   
4               (B09NQ44RNV)              (B09KXT4VG7)            0.234694   
5               (B09KXT4VG7)              (B09NQ44RNV)            0.020408   
6               (B09NQ44RNV)              (B09NDKQD88)            0.234694   
7               (B09NDKQD88)              (B09NQ44RNV)            0.030612   
8               (B09SDXFFQ1)              (B09SDX13G7)            0.326531   
9               (B09SDX13G7)              (B09SDXFFQ1)            0.040816   
10              (B09SDXFFQ1)              (B09SDXG1CP)            0.326531   
11              (B09SDXG1CP)              (B09SDXFFQ1)       

In [166]:
interesting_rules = new_rules[(new_rules['lift']> 1) & (new_rules['confidence'] >= 0.5)]
print(interesting_rules.shape[0])
print(interesting_rules.head())

24
    antecedents   consequents  antecedent support  consequent support  \
1  (B081X4S825)  (B09NQ44RNV)            0.020408            0.234694   
2  (B0894Y4PNG)  (B09NQ44RNV)            0.020408            0.234694   
5  (B09KXT4VG7)  (B09NQ44RNV)            0.020408            0.234694   
7  (B09NDKQD88)  (B09NQ44RNV)            0.030612            0.234694   
9  (B09SDX13G7)  (B09SDXFFQ1)            0.040816            0.326531   

    support  confidence      lift  leverage  conviction  zhangs_metric  
1  0.020408        1.00  4.260870  0.015618         inf       0.781250  
2  0.020408        1.00  4.260870  0.015618         inf       0.781250  
5  0.020408        1.00  4.260870  0.015618         inf       0.781250  
7  0.030612        1.00  4.260870  0.023428         inf       0.789474  
9  0.030612        0.75  2.296875  0.017284    2.693878       0.588652  


In [167]:
interesting_rules.drop(columns = ['antecedent support', 'consequent support', 'conviction', 'leverage'])
interesting_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
1,(B081X4S825),(B09NQ44RNV),0.020408,0.234694,0.020408,1.000000,4.260870,0.015618,inf,0.781250
2,(B0894Y4PNG),(B09NQ44RNV),0.020408,0.234694,0.020408,1.000000,4.260870,0.015618,inf,0.781250
5,(B09KXT4VG7),(B09NQ44RNV),0.020408,0.234694,0.020408,1.000000,4.260870,0.015618,inf,0.781250
7,(B09NDKQD88),(B09NQ44RNV),0.030612,0.234694,0.030612,1.000000,4.260870,0.023428,inf,0.789474
9,(B09SDX13G7),(B09SDXFFQ1),0.040816,0.326531,0.030612,0.750000,2.296875,0.017284,2.693878,0.588652
14,(B09SDXWNPZ),(B09SDXFFQ1),0.040816,0.326531,0.040816,1.000000,3.062500,0.027489,inf,0.702128
17,(B09SDY51NS),(B09SDXFFQ1),0.061224,0.326531,0.051020,0.833333,2.552083,0.031029,4.040816,0.647826
19,(B09SDZ4FH9),(B09SDXFFQ1),0.030612,0.326531,0.030612,1.000000,3.062500,0.020616,inf,0.694737
21,(B09SDXG1CP),(B09SDXRYBG),0.051020,0.214286,0.030612,0.600000,2.800000,0.019679,1.964286,0.677419
22,(B09SDXQ3X9),(B09SDY8DCT),0.071429,0.265306,0.040816,0.571429,2.153846,0.021866,1.714286,0.576923
